In [1]:
include("comum.jl")
include("modelos.jl")
using Evolutionary, Plots;
pyplot();

In [2]:
Ts = 0.08
tend = 2.0
t0 = 0.0
popul  = 50   # população
iterac = 3 #iterações
xr = SVector{7}([1.,1.,0.5,0.6,0.5,1.2,0.8]);

In [3]:
function gerador(n)
    rand(n).*rand([10.,100.,1000.,10000],n)
end;

In [4]:
function custo(gain::Vector{Float64})
    kp = SMatrix{7,7}(diagm(gain[1:7]))
    kv = SMatrix{7,7}(diagm(gain[8:end]))
    x, v, t, a, ta, j, tj = kukaRobot(kp, kv, Ts, t0, tend, xr)
    sizeVector = length(x[1])

    erro_sum = 0.
    for i=1:7
        erro = -(x[i]-xr[i])
        erro_sum = erro_sum + sum(abs.(erro[floor(Integer,sizeVector/3):end])) 
    end

    jerk_sum = 0.
    for i=1:7
        jerk_sum = jerk_sum + sum(abs.(j[i]))
    end

    
    erro_sum = erro_sum * 10.
    jerk_sum = jerk_sum * 0.01

    out =  erro_sum + jerk_sum
    println("$(erro_sum) + $(jerk_sum)")
    out
end;

In [ ]:
N = 14
result, fitness, cnt = ga(custo, N; initPopulation = gerador, populationSize = popul, ɛ = 0.25, selection = roulette, crossover = intermediate(0.25), mutation = domainrange(fill(0.5,N)), iterations = iterac)

In [ ]:
t_end_new = tend + 2
kp = SMatrix{7,7}(diagm(result[1:7]))
kv = SMatrix{7,7}(diagm(result[8:end]))

x, v, t, a, ta, j, tj = kukaRobot(kp, kv, Ts, t0, t_end_new, xr);

In [ ]:
for i=1:7
    erro = xr[i] - x[i][end]
    println("erro final $(i) = $(erro)")
end   

In [ ]:
function plotx(i)
    p1 = plot(t,x[i], label = "desejado")
    p1= plot!([xr[i]],seriestype= :hline, label = "referência");
end    

In [ ]:
plotx(2)